In [4]:
#Task1 Problem 1

#Divide and conquer matrix algo 

import numpy as np

def split_matrix(matrix):
    """
    Splits a given n x n matrix into four submatrices of size n/2 x n/2.
    """
    n = matrix.shape[0]
    mid = n // 2
    return matrix[:mid, :mid], matrix[:mid, mid:], matrix[mid:, :mid], matrix[mid:, mid:]

def naive_recursive_multiply(A, B):
    """
    Recursively multiplies two matrices A and B using divide-and-conquer.
    """
    n = A.shape[0]

    #Base case: if the matrix is 1x1 :return the scalar multiplication
    if n == 1:
        return A * B

    #splits the matrices into submatrices
    A11, A12, A21, A22 = split_matrix(A)
    B11, B12, B21, B22 = split_matrix(B)

    #computes the 8 submatrix products recursively
    M1 = naive_recursive_multiply(A11, B11)
    M2 = naive_recursive_multiply(A12, B21)
    M3 = naive_recursive_multiply(A11, B12)
    M4 = naive_recursive_multiply(A12, B22)
    M5 = naive_recursive_multiply(A21, B11)
    M6 = naive_recursive_multiply(A22, B21)
    M7 = naive_recursive_multiply(A21, B12)
    M8 = naive_recursive_multiply(A22, B22)

    #Combine the results 
    C11 = M1 + M2
    C12 = M3 + M4
    C21 = M5 + M6
    C22 = M7 + M8

    #reconstruct the full matrix from submatrices
    top = np.hstack((C11, C12))
    bottom = np.hstack((C21, C22))
    return np.vstack((top, bottom))


if __name__ == "__main__":
    
    A = np.array([[1, 2, 3, 4],
                  [5, 6, 7, 8],
                  [9, 10, 11, 12],
                  [13, 14, 15, 16]])

    B = np.array([[16, 15, 14, 13],
                  [12, 11, 10, 9],
                  [8, 7, 6, 5],
                  [4, 3, 2, 1]])

    
    C = naive_recursive_multiply(A, B)

 
    print("Result of matrix multiplication:")
    print(C)
#T(n) = aT(n/b) + f(n), a = # of sub problems, b = factor divided by 

Result of matrix multiplication:
[[ 80  70  60  50]
 [240 214 188 162]
 [400 358 316 274]
 [560 502 444 386]]


In [3]:
# Task 1 Q2 
import numpy as np

def split_matrix(matrix):
    """
    Splits a given n x n matrix into four submatrices of size n/2 x n/2.
    """
    n = matrix.shape[0]
    mid = n // 2
    return matrix[:mid, :mid], matrix[:mid, mid:], matrix[mid:, :mid], matrix[mid:, mid:]

def strassen_recursive_multiply(A, B):
    """
    Recursively multiplies two matrices A and B using Strassen's algorithm.
    """
    n = A.shape[0]

    
    if n == 1:
        return A * B

    
    A11, A12, A21, A22 = split_matrix(A)
    B11, B12, B21, B22 = split_matrix(B)

    # 7 products using Strassen's formula
    M1 = strassen_recursive_multiply(A11 + A22, B11 + B22)
    M2 = strassen_recursive_multiply(A21 + A22, B11)
    M3 = strassen_recursive_multiply(A11, B12 - B22)
    M4 = strassen_recursive_multiply(A22, B21 - B11)
    M5 = strassen_recursive_multiply(A11 + A12, B22)
    M6 = strassen_recursive_multiply(A21 - A11, B11 + B12)
    M7 = strassen_recursive_multiply(A12 - A22, B21 + B22)

    
    C11 = M1 + M4 - M5 + M7
    C12 = M3 + M5
    C21 = M2 + M4
    C22 = M1 - M2 + M3 + M6

    
    top = np.hstack((C11, C12))
    bottom = np.hstack((C21, C22))
    return np.vstack((top, bottom))


if __name__ == "__main__":
    
    A = np.array([[1, 2, 3, 4],
                  [5, 6, 7, 8],
                  [9, 10, 11, 12],
                  [13, 14, 15, 16]])

    B = np.array([[16, 15, 14, 13],
                  [12, 11, 10, 9],
                  [8, 7, 6, 5],
                  [4, 3, 2, 1]])

    
    C = strassen_recursive_multiply(A, B)

    
    print("Result of matrix multiplication using Strassen's algorithm:")
    print(C)
#a=7 b = 2 f(n) = O(n^2)

Result of matrix multiplication using Strassen's algorithm:
[[ 80  70  60  50]
 [240 214 188 162]
 [400 358 316 274]
 [560 502 444 386]]


In [5]:
#Task 2 Q1

import numpy as np
from scipy.sparse import dok_matrix, csr_matrix
from scipy.sparse.linalg import eigsh

def construct_hamiltonian(N, J=1.0, h=1.0):
    """
    Constructs the Hamiltonian matrix for a spin-1/2 chain with periodic boundary conditions.

    Parameters:
        N (int): Number of chain elements (spins).
        J (float): Coupling constant for the interaction term.
        h (float): Magnetic field strength for the transverse field term.

    Returns:
        H (csr_matrix): Sparse Hamiltonian matrix in the computational basis.
    """
    #total size of the Hilbert space 
    dim = 2**N

    #creates a sparse matrix using dok format 
    H = dok_matrix((dim, dim), dtype=np.float64)

    #compute spin basis states
    for state in range(dim):
        
        for i in range(N):
            #determines the spins at site i and i+1 with periodic boundary
            spin_i = (state >> i) & 1
            spin_ip1 = (state >> ((i + 1) % N)) & 1

            #Sz_i * Sz_(i+1) gives +1 for aligned spins, -1 for anti-aligned
            sz_sz = (1 if spin_i == spin_ip1 else -1)
            H[state, state] += -J * sz_sz

        #transverse field term: -h * Sx_i
        for i in range(N):
            #flip the spin at site i 
            flipped_state = state ^ (1 << i)
            H[state, flipped_state] += -h

    #Convert to CSR format 
    return H.tocsr()


if __name__ == "__main__":
    N = 4  # Number of spins in the chain
    J = 1.0  # Coupling constant
    h = 0.5  # Transverse field strength

   
    H = construct_hamiltonian(N, J, h)

    #computes the lowest eigenvalue using sparse eigensolver
    eigenvalues, eigenvectors = eigsh(H, k=1, which='SA')

   
    print("Ground state energy:", eigenvalues[0])


Ground state energy: -4.271558410139712


In [7]:
#Task 2 Q2 

import numpy as np
from scipy.linalg import qr

def qr_algorithm(H, max_iterations=1000, tol=1e-10):
    """
    Applies the QR algorithm to compute the diagonal form of a given matrix H.

    Parameters:
        H (ndarray): Input Hamiltonian matrix (assumed square).
        max_iterations (int): Maximum number of iterations for convergence.
        tol (float): Convergence tolerance for off-diagonal elements.

    Returns:
        D (ndarray): Diagonalized matrix (approximately).
    """
    
    A = H.copy()

    for _ in range(max_iterations):
        
        Q, R = qr(A)

        #updates the matrix
        A_next = R @ Q

        #convergence check 
        if np.allclose(A, A_next, atol=tol):
            break

        A = A_next

    return np.diag(np.diag(A))  #return diagonal matrix


if __name__ == "__main__":
    
    H = np.array([[4, 1, 2],
                  [1, 3, 1],
                  [2, 1, 3]], dtype=float)

    
    D = qr_algorithm(H)

    
    print("Diagonal matrix:")
    print(D)



Diagonal matrix:
[[6.18194334 0.         0.        ]
 [0.         2.40642065 0.        ]
 [0.         0.         1.41163601]]
